In [1]:
import os
import time
import re
from PIL import Image
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
from selenium import webdriver

# 무신사 전체 카테고리 id 크롤링

In [2]:
# path = "/root/chrome_driver/chromedriver"
path_window = r"/Users/seungsu/Desktop/YBIGTA/chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1920x1080")
# chrome_options.add_argument("headless")

driver = webdriver.Chrome(path_window, options=chrome_options)
driver.implicitly_wait(3)

In [3]:
driver.get("https://store.musinsa.com/app/")

In [4]:
all_nav = driver.find_element_by_css_selector("nav.nav_menu")
navs = all_nav.find_elements_by_css_selector("div.nav_category")
category = {}
for nav in navs[1:]:
    cat = {}
    big = nav.find_element_by_css_selector("div.nav_menu_title").find_element_by_css_selector("a")
    big.click()
    big_cat_name = big.find_element_by_css_selector("strong.title").text
    ll = nav.find_element_by_css_selector("div.item_sub_menu")
    uls = ll.find_elements_by_css_selector("ul")
    for ul in uls:
        al = ul.find_elements_by_css_selector("li")
        for a in al:
            m = a.find_element_by_css_selector("a")
            h = m.get_attribute("href")
            #
            exp = re.compile('[가-힣\s/]*')
            title = exp.findall(m.text)[0].strip(" ")
            #
            cat[title] = h
    
    category[big_cat_name] = cat
driver.close()
print(category)

{'상의': {'반팔 티셔츠': 'https://store.musinsa.com/app/items/lists/001001', '긴팔 티셔츠': 'https://store.musinsa.com/app/items/lists/001010', '민소매 티셔츠': 'https://store.musinsa.com/app/items/lists/001011', '셔츠/블라우스': 'https://store.musinsa.com/app/items/lists/001002', '피케/카라 티셔츠': 'https://store.musinsa.com/app/items/lists/001003', '맨투맨/스웨트셔츠': 'https://store.musinsa.com/app/items/lists/001005', '후드 스웨트셔츠/후드 집업': 'https://store.musinsa.com/app/items/lists/001004', '니트/스웨터/카디건': 'https://store.musinsa.com/app/items/lists/001006', '베스트': 'https://store.musinsa.com/app/items/lists/001012', '기타 상의': 'https://store.musinsa.com/app/items/lists/001008'}, '아우터': {'블루종/': 'https://store.musinsa.com/app/items/lists/002001', '레더/라이더스 재킷': 'https://store.musinsa.com/app/items/lists/002002', '트러커 재킷': 'https://store.musinsa.com/app/items/lists/002017', '수트/블레이저 재킷': 'https://store.musinsa.com/app/items/lists/002003', '나일론/코치/아노락 재킷': 'https://store.musinsa.com/app/items/lists/002006', '스타디움 재킷': 'https://stor

In [7]:
print(category.keys())

dict_keys(['상의', '아우터', '원피스', '바지', '스커트', '가방', '스니커즈', '신발', '시계', '모자', '스포츠/용품', '레그웨어/속옷', '안경', '액세서리', '디지털/테크', '생활/취미/예술', '뷰티', '반려동물', '책/음악/티켓'])


In [14]:
print(category["바지"])

{'데님 팬츠': 'https://store.musinsa.com/app/items/lists/003002', '코튼 팬츠': 'https://store.musinsa.com/app/items/lists/003007', '수트 팬츠/슬랙스': 'https://store.musinsa.com/app/items/lists/003008', '트레이닝/조거 팬츠': 'https://store.musinsa.com/app/items/lists/003004', '숏 팬츠': 'https://store.musinsa.com/app/items/lists/003009', '레깅스': 'https://store.musinsa.com/app/items/lists/003005', '기타 바지': 'https://store.musinsa.com/app/items/lists/003006'}


# 디렉토리 생성

In [7]:
def check_directory(category_name, folder_path=None):
    if folder_path == None:
        folder_path = "C:/Users/seungsu/Desktop/ICT-project/crawling"
    try:
        os.mkdir(f"{folder_path}/{category_name}/")
        print("Directory is created!")
    except:
        print("Directory exist!")

## 페이지 Bsobj받아오기 (+header)

In [8]:
def getBSobj(page="1",category_id="001004"):
    url = f"https://store.musinsa.com/app/items/lists/{category_id}/?category=&d_cat_cd={category_id}&u_cat_cd=&brand=&sort=pop&sub_sort=&display_cnt=90&page={page}&page_kind=category&list_kind=small&free_dlv=&ex_soldout=&sale_goods=&exclusive_yn=&price=&color=&a_cat_cd=&sex=&size=&tag=&popup=&brand_favorite_yn=&goods_favorite_yn=&blf_yn=&campaign_yn=&price1=&price2=&brand_favorite=&goods_favorite=&chk_exclusive=&chk_sale=&chk_soldout="
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content
    bsObj = bs(html, "html.parser")
    return bsObj

## 단일 페이지 크롤러

In [9]:
def crawl_page(bsObj, category_name):
    body = bsObj.find("ul", {"class":"snap-article-list"})
    all_items = body.find_all("li", {"class":"li_box"})
    for li in all_items:
        a = li.find("a", {"class":"img-block"})
        id = a["href"].lstrip("/app/product/detail/").rstrip("/0")
        img = "http:" + a.find("img")["data-original"]
        image = urllib.request.urlretrieve(img, f"./{category_name}/{id}.png")
        #image resize
        image = Image.open(f"./{category_name}/{id}.png")
        resize_img = image.resize((125,125))
        resize_img.save(f"./{category_name}/{id}.png")
        
# crawl_page(getBSobj(page="1",category="001004"), category_name="상의")

## 서브 카테고리 크롤러

In [10]:
def crawl_SubCategory(main='상의', sub='반팔 티셔츠', page=3):
    #전역변수 category
    category_id = category[main][sub].lstrip("https://store.musinsa.com/app/items/lists/")
    for p in range(1, int(page+1)):
        print(f"CRAWLING\t{main}\t>\t{sub} \t\t page = {p}")
        bsObj = getBSobj(page=str(p),category_id=category_id)
        crawl_page(bsObj, category_name=main)
        time.sleep(2)
        

In [11]:
# check_directory("상의")
# crawl_SubCategory()

## 전체 카테고리 크롤러

In [13]:
def crawl_AllCategory(categories=None, page=1, folder_path=None):
    if categories == None:
        for category_name, contents in category.items():
            check_directory(category_name=category_name, folder_path=folder_path)
            for sub_category_name,n in contents.items():
                crawl_SubCategory(main=category_name, sub=sub_category_name, page=page)
                print(f"CRAWLING COMPLETE\t{category_name} > {sub_category_name}")
    else:
        # categories는 category 이름 리스트로 받는다
        for category_name in categories:
            check_directory(category_name=category_name, folder_path=folder_path)
            for sub_category_name,n in category[category_name].items():
                crawl_SubCategory(main=category_name, sub=sub_category_name, page=page)
                print(f"CRAWLING COMPLETE\t{category_name} > {sub_category_name}")
            
crawl_AllCategory(categories=["바지", "아우터"])

Directory is created!
CRAWLING	바지	>	데님 팬츠 		 page = 1
CRAWLING COMPLETE	바지 > 데님 팬츠
CRAWLING	바지	>	코튼 팬츠 		 page = 1
CRAWLING COMPLETE	바지 > 코튼 팬츠
CRAWLING	바지	>	수트 팬츠/슬랙스 		 page = 1
CRAWLING COMPLETE	바지 > 수트 팬츠/슬랙스
CRAWLING	바지	>	트레이닝/조거 팬츠 		 page = 1
CRAWLING COMPLETE	바지 > 트레이닝/조거 팬츠
CRAWLING	바지	>	숏 팬츠 		 page = 1
CRAWLING COMPLETE	바지 > 숏 팬츠
CRAWLING	바지	>	레깅스 		 page = 1
CRAWLING COMPLETE	바지 > 레깅스
CRAWLING	바지	>	기타 바지 		 page = 1
CRAWLING COMPLETE	바지 > 기타 바지
Directory is created!
CRAWLING	아우터	>	블루종/ 		 page = 1
CRAWLING COMPLETE	아우터 > 블루종/
CRAWLING	아우터	>	레더/라이더스 재킷 		 page = 1
CRAWLING COMPLETE	아우터 > 레더/라이더스 재킷
CRAWLING	아우터	>	트러커 재킷 		 page = 1
CRAWLING COMPLETE	아우터 > 트러커 재킷
CRAWLING	아우터	>	수트/블레이저 재킷 		 page = 1
CRAWLING COMPLETE	아우터 > 수트/블레이저 재킷
CRAWLING	아우터	>	나일론/코치/아노락 재킷 		 page = 1
CRAWLING COMPLETE	아우터 > 나일론/코치/아노락 재킷
CRAWLING	아우터	>	스타디움 재킷 		 page = 1
CRAWLING COMPLETE	아우터 > 스타디움 재킷
CRAWLING	아우터	>	환절기 코트 		 page = 1
CRAWLING COMPLETE	아우터 > 환절기 코트
CRAWLING	아우터	>	겨울 싱글 코트 		 page = 

In [6]:
urllib.request.urlretrieve("http://image.msscdn.net/images/goods_img/20200401/1378259/1378259_1_125.jpg", f"./{id}.png")

OSError: [Errno 22] Invalid argument: './<built-in function id>.png'